One way to represent the text is to convert sentences into embeddings vectors. We can use a pre-trained text embedding as the first layer, which will have two advantages:

we don't have to worry about text preprocessing,
we can benefit from transfer learning.
Using Model --> TensorFlow Hub called google/tf2-preview/gnews-swivel-20dim/1.




Set Tensorflow Version

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


! is used because its a command line command and not a python command


In [ ]:
!pip install tensorflow

In [ ]:

import tensorflow as tf 
print(tf.__version__)

2.1.0


Read in Data and allocate to test and training sets

In [ ]:
import pandas as pd
from google.colab import files
#files.upload()                           #Uncomment this to enable in browser file upload
training_data = pd.read_csv("training_data.csv",sep='|',header=None)
test_data = pd.read_csv("test_data.csv",sep='|',header=None)

#remove headers
training_data[0].pop(0)
training_data[1].pop(0)
test_data[0].pop(0)
test_data[1].pop(0)

#questions
print("Before Shuffle - questions")
for a in training_data[0][:10]:
    print(a)

#answers
print("Before Shuffle - answers")
for a in training_data[1][:10]:
    print(a)

#Shuffle input
training_data = training_data.sample(frac=1).reset_index(drop=True)

#questions
print("After Shuffle - questions")
for a in training_data[0][:10]:
    print(a)

#answers
print("After Shuffle - answers")
for a in training_data[1][:10]:
    print(a)




In [ ]:
#split into Training, Validation and Test.
train_examples = training_data[0]
train_labels = training_data[1]

test_examples = test_data[0]
test_labels = test_data[1]

In [ ]:
#Convert Labels Catagories into
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit_transform(train_labels)
train_labels_int = encoder.transform(train_labels)
test_labels_int = encoder.transform(test_labels)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)

for a in train_labels[:10]:
    print(a)

for a in train_labels_int[:10]:
    print(a)


In [ ]:
#import existing trained NLP Model
#converts from a sentence into a 20 dimention vector.
import tensorflow_hub as hub

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])



<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 0.7767494 , -1.3949643 ,  1.8822532 ,  0.5356442 , -1.0081346 ,
        -2.2011378 , -0.04300665,  0.27778423, -0.26042593,  1.3977667 ,
        -0.03493638, -0.74132186, -0.08866705,  0.6111342 , -2.9438055 ,
        -0.3955079 ,  3.248432  , -0.95043993, -1.321964  , -0.00502632],
       [ 0.371193  , -1.1949918 ,  1.9071492 ,  0.36187264,  0.24854447,
        -1.595161  ,  0.5150494 , -0.65282017, -0.7913214 ,  0.66825783,
         0.68587404, -0.6499917 , -0.14705275,  0.80869836, -2.0821605 ,
        -0.71447015,  3.2234712 , -1.6162708 , -0.9779024 ,  0.20788188],
       [ 0.712013  , -1.6389797 ,  1.901473  ,  1.5259099 , -1.5908127 ,
        -2.4464293 ,  0.09874652,  0.7626246 , -0.27520013,  0.15411198,
        -0.9386238 ,  0.3088036 ,  0.02570286,  0.84749335, -0.97850776,
        -0.217274  ,  2.667848  , -0.39907122, -0.29446137, -1.3394673 ]],
      dtype=float32)>

In [ ]:
#Build the rest of the model
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))         #1 hidden layer
model.add(tf.keras.layers.Dense(5, activation='softmax'))     #was sigmoid

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 20)                400020    
_________________________________________________________________
dense_2 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 85        
Total params: 400,441
Trainable params: 400,441
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Create a validation Set
x_val = train_examples[:50]
x_train = train_examples#[50:]

y_val = train_labels_int[:50]
y_train = train_labels_int#[50:]

test_examples = test_data[0]
test_labels = test_data[1]

import numpy as np
y_val = np.array(y_val)
x_val = np.array(x_val)
y_train = np.array(y_train)
x_train = np.array(x_train)
test_examples = np.array(test_examples)
test_labels =  np.array(test_labels)

print ("x validation: " , len(x_val))
print ("x train: " , len(x_train))
print ("y validation: " , len(y_val))
print ("y train: " , len(y_train))
print ("x test: " , len(test_examples))
print ("y test: " , len(test_labels_int))



x validation:  50
x train:  280
y validation:  50
y train:  280
x test:  21
y test:  21


In [ ]:
#set remaining parameters
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=50,
                    validation_data=(x_val, y_val),verbose=1)

Train on 280 samples, validate on 50 samples
Epoch 1/40
280/280 [==============================] - 0s 2ms/sample - loss: 0.0301 - accuracy: 0.9964 - val_loss: 0.0166 - val_accuracy: 1.0000
Epoch 2/40
280/280 [==============================] - 0s 188us/sample - loss: 0.0262 - accuracy: 0.9964 - val_loss: 0.0132 - val_accuracy: 1.0000
Epoch 3/40
280/280 [==============================] - 0s 235us/sample - loss: 0.0233 - accuracy: 0.9964 - val_loss: 0.0107 - val_accuracy: 1.0000
Epoch 4/40
280/280 [==============================] - 0s 185us/sample - loss: 0.0209 - accuracy: 0.9964 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 5/40
280/280 [==============================] - 0s 201us/sample - loss: 0.0188 - accuracy: 0.9964 - val_loss: 0.0079 - val_accuracy: 1.0000
Epoch 6/40
280/280 [==============================] - 0s 197us/sample - loss: 0.0171 - accuracy: 0.9964 - val_loss: 0.0067 - val_accuracy: 1.0000
Epoch 7/40
280/280 [==============================] - 0s 184us/sample - loss: 0.0

In [ ]:
#Evaluate on Test Data
results = model.evaluate(test_examples, test_labels_int)

print(results)

21/21 [==============================] - 0s 4ms/sample - loss: 0.0027 - accuracy: 1.0000
[0.0026799559127539396, 1.0]


In [ ]:
import matplotlib.pyplot as plt

def bar_graph(x1,y1):

    probabilities = model.predict([x1])

    #pred = tf.keras.backend.argmax(probabilities,1)
    prediction = np.argmax(probabilities[0])
    y=probabilities[0]
    N = len(probabilities[0])
    x = range(N)
    width = 1/1.5
    plt.bar(x, y, width, color="blue")
    plt.xticks(x,encoder.classes_)
    plt.show()
    print ("Input:",x1)
    print ("Actual:",encoder.classes_[y1], " Predicted: ", encoder.classes_[prediction])

for i in range(len(test_examples)):
  bar_graph(test_examples[i],test_labels_int[i])


In [ ]:
# Other Experiments
import nltk 
nltk.download('punkt')
tokens = nltk.word_tokenize("The Quick Brown Fox, Jumps over the lazy dog dog dog dog")
print(tokens)
unique_words = set(tokens)
print(unique_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
['The', 'Quick', 'Brown', 'Fox', ',', 'Jumps', 'over', 'the', 'lazy', 'dog', 'dog', 'dog', 'dog']
{'The', 'lazy', 'Quick', 'dog', 'the', 'Brown', 'Fox', 'over', 'Jumps', ','}
